# Base de datos de Programas de tv

Este script genera una base datos con las siguientes columnas:
    
- **title:** título del programa de televisión
- **network:** cadena de televisión.
- **country:** país de la productora del programa de televisión.
- **classification:** clasificación del programa con base en la edad.
- **start_date:** fecha de transmisión del primer capitulo.
- **seasons:** número de temporadas.
- **status:** estatus del programa (en desarrollo, en transmisión, terminado)
- **rating:** rating del show según IMDb 
- **votes:** número de votos
- **cast:** actores del programa de televisión
- **description:** breve descripción de la trama del programa
- **+15 columnas dummies:** 'TV Comedies', 'Crime TV Shows', 'TV Dramas', 'Docuseries', 'Teen TV Shows', 'TV Horror', "Kids' TV", etc.


A partir de la extracción de 3 fuentes de información:

1. **Archivo csv:** https://www.kaggle.com/shivamb/netflix-shows?select=netflix_titles.csv
2. **API:** https://www.episodate.com/api
3. **Web scraping:** https://www.imdb.com/search/title/?title_type=tv_series

## Librerías

In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import urllib

## DATA CSV

### Descripción 
El archivo netflix_titles.csv se encuentra en: https://www.kaggle.com/shivamb/netflix-shows?select=netflix_titles.csv este archivo contiene 6,234 registros de programas de televisión y películas con 12 columnas:

##### Descripción de las columnas
    
01. **show_id:** Id único para cada película / serie de televisión
02. **type:** película /  serie de televisión
03. **title:** título de la película
04. **director:** director de la película
05. **cast:** actores de la película / serie de televisión
06. **country:** país donde la película / serie de televisión fue producida.
07. **date_added:** fecha en la que fue agregada a Netflix.
08. **release_year:** año de lanzamiento.
09. **rating:** clasificación por edad.
10. **duration:** duración de la película o número de temporadas
11. **listed_in:** género.
12. **description:** breve descripción

### Código lectura

In [2]:
# Lectura del archivo
path = 'C:/Users/Itzel/Downloads/netflix_titles.csv'
df_csv = pd.read_csv(path)

In [3]:
# Visualizar rasgos generales
print(df_csv.shape)
df_csv.head(5)

(6234, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


Seleccionamos unicamente los registros de tipo TV Show y las columnas: title, director, cast, duration, rating, listed_in, description. Obteniendo XXX  registros.

In [4]:
# Filtramos por TV Show
df_csv = df_csv[ df_csv.type == 'TV Show']

# Seleccionar columnas
df_csv = df_csv [['title', 'director', 'cast', 'duration', 'rating', 'listed_in', 'description']]

print('Total de tv shows:', df_csv.shape[0])

Total de tv shows: 1969


In [5]:
# Renombrar columnas
df_csv.rename(columns={'rating': 'classification'}, inplace = True)

In [24]:
print('Títulos unicos', len(df_csv.title.unique()))

Títulos unicos 1958


In [33]:
# Quitamos duplicados sobre el título
df_csv.drop_duplicates(['title'], inplace=True)

## DATA API

### Descripción del archivo

Los datos obtenidos a través de la api, se encuentran en https://www.episodate.com/api tomando el endpoint _/most-popular_
la cual ofrece una lista completa de los programas de televisión más populares con la siguiente información:

#### Descripción de las columnas

1. **id:** id único del programa de televisión.
2. **name:** título del programa de televisión.
3. **permalink:** título separado por -
4. **start_date:** fecha de inicio de transmisión del primer capitulo.
5. **end_date:** fecha de transmisión del último capitulo.
6. **country**: país de la productora del programa de televisión.
7. **network:** cadena de televisión.
8. **status:** estatus del programa de televisión.
9. **image_thumbnail_path:** path 

### Código extracción

In [6]:
# Url
url = 'https://www.episodate.com/api/most-popular'
req = requests.get(url)
req

<Response [200]>

In [7]:
api = req.json()

# Exploración de las llaves
print(api.keys())

dict_keys(['total', 'page', 'pages', 'tv_shows'])


In [8]:
# Exploración del archivo json
print(api)

{'total': '17130', 'page': 1, 'pages': 857, 'tv_shows': [{'id': 35624, 'name': 'The Flash', 'permalink': 'the-flash', 'start_date': '2014-10-07', 'end_date': None, 'country': 'US', 'network': 'The CW', 'status': 'Running', 'image_thumbnail_path': 'https://static.episodate.com/images/tv-show/thumbnail/35624.jpg'}, {'id': 23455, 'name': 'Game of Thrones', 'permalink': 'game-of-thrones', 'start_date': '2011-04-17', 'end_date': None, 'country': 'US', 'network': 'HBO', 'status': 'Ended', 'image_thumbnail_path': 'https://static.episodate.com/images/tv-show/thumbnail/23455.jpg'}, {'id': 29560, 'name': 'Arrow', 'permalink': 'arrow', 'start_date': '2012-10-10', 'end_date': None, 'country': 'US', 'network': 'The CW', 'status': 'Ended', 'image_thumbnail_path': 'https://static.episodate.com/images/tv-show/thumbnail/29560.jpg'}, {'id': 43467, 'name': 'Lucifer', 'permalink': 'lucifer', 'start_date': '2016-01-25', 'end_date': None, 'country': 'US', 'network': 'Netflix', 'status': 'Running', 'image_th

Al revisar el contenido de la API observamos que existen un total de 17130 de registros distribuidos en 857 páginas.
Para recolectar toda la data, tomaremos la siguiente url más el número de página:
    
    https://www.episodate.com/api/most-popular?page= + número de página
    
Por ejemplo:
    https://www.episodate.com/api/most-popular?page=1

In [9]:
%%time
# Código para extraer la información

# Url
api_url = 'https://www.episodate.com/api/most-popular?page='

# df que guardará la información de la api
df_api = pd.DataFrame()

# Iterar sobre todas las páginas
for i in range(1,858):
    try:                
        req = requests.get(api_url + str(i))
        req_json = req.json()
        tv_shows = req_json['tv_shows']
        df_api = pd.concat([df_api, pd.DataFrame(tv_shows)])
    except:
        pass



Wall time: 1h 10min 55s


In [10]:
df_api.head()

,id,name,permalink,start_date,end_date,country,network,status,image_thumbnail_path
0,35624,The Flash,the-flash,2014-10-07,None,US,The CW,Running,https://static.episodate.com/images/tv-show/th...
1,23455,Game of Thrones,game-of-thrones,2011-04-17,None,US,HBO,Ended,https://static.episodate.com/images/tv-show/th...
2,29560,Arrow,arrow,2012-10-10,None,US,The CW,Ended,https://static.episodate.com/images/tv-show/th...
3,43467,Lucifer,lucifer,2016-01-25,None,US,Netflix,Running,https://static.episodate.com/images/tv-show/th...
4,43234,Supergirl,supergirl,2015-10-26,None,US,The CW,Running,https://static.episodate.com/images/tv-show/th...


In [11]:
# Seleccionamos las columnas deseadas
df_api =  df_api[['name', 'start_date', 'end_date', 'country', 'network', 'status']]

In [12]:
# Renombramos
df_api.rename(columns={'name': 'title'}, inplace = True)

In [35]:
print('Total de títulos:', df_api.shape[0])
print('Títulos unicos', len(df_api.title.unique()))

Total de títulos: 17126
Títulos unicos 16327


In [41]:
# quitando duplicados
df_api.drop_duplicates(['title'], inplace=True)

## DATA WEB SCRAPING

### Descripción

Por medio de web scraping https://www.imdb.com/search/title/?title_type=tv_series se obtiene la siguiente información acerca del rating según IMDb:

#### Descripción de las columnas:
1. **title:** título de la serie de televisión
2. **rating:** rating del show según IMDb
3. **votes:** número de votos


### Código de extracción

In [64]:
url2 = 'https://www.imdb.com/search/title/?title_type=tv_series'
req =  requests.get(url2)

# Verificar respuesta
req

<Response [200]>

In [65]:
sopa =  BeautifulSoup(req.content, 'html.parser')

# Filtrar sopa por el tag que nos interesa
lst_sopa= sopa.findAll('div', {'class': 'lister-item-content'})

In [66]:
# Imprimir longitud de la lista de sopas
len(lst_sopa)

50

In [67]:
# # Explorar lista de sopas
# lst_sopa

In [68]:
###########################################################################################################
################################        Funciones      ####################################################
###########################################################################################################

def data_web(lst_sopa):
    """Función que recibe como argumento una lista de sopa del url, regresa 3 listas:
    titles, ratings, votes
    """
    titles = []

    #Ratings
    ratings = []

    # Votes
    votes = []
    for i in range(len(lst_sopa)):

        try:
            # Títulos
            title = re.findall('href="\/title\/.*', str(lst_sopa[i]))[0]
            title_clean = re.findall('>.*?<', title)[0].replace('<', '').replace('>', '')

            # Rating
            rating = lst_sopa[i].findAll('div',  {'class': 'inline-block ratings-imdb-rating'})[0]
            rating_clean = rating.text.replace('\n', '')

            # Votes
            vote = lst_sopa[i].findAll('span')[-1].text
            vote_clean = vote.replace(',', '')

            # Agregar resultados        
            titles.append(title_clean)
            ratings.append(rating_clean)
            votes.append(vote_clean)

        except:
            pass
    return titles, ratings, votes

Al analizar la información de la url, se observa que viene la información por página registros **1-50 of 193,919 titles**, al dar click a la siguiente página, el url se actualiza a https://www.imdb.com/search/title/?title_type=tv_series&start=51&ref_=adv_nxt por lo que se iterará sobre el número 51 para extraer la información de al menos 15,000 registros aproximadamente.

In [69]:
%%time

# Contador para detener el ciclo while cuando se tengan más de 15,000 registros
n_data = 0

# df que guardará la información obtenida a través de web scraping
df_web =pd.DataFrame(columns=['title', 'rating', 'votes'])


# Contador auxiliar para cambiar de url
n_url=1 

while n_data <= 15000:
    
    # Condición if para la primera iteración
    if n_data == 0:
        url2 = 'https://www.imdb.com/search/title/?title_type=tv_series'
        req =  requests.get(url2)
        sopa =  BeautifulSoup(req.content, 'html.parser')
        lst_sopa= sopa.findAll('div', {'class': 'lister-item-content'})

    # Para el resto de las iteraciones
    else:
        url3 = 'https://www.imdb.com/search/title/?title_type=tv_series&start='+str(n_url)+'&ref_=adv_nxt'
        req =  requests.get(url3)
        sopa =  BeautifulSoup(req.content, 'html.parser')
        lst_sopa= sopa.findAll('div', {'class': 'lister-item-content'})        
    
    # Extraer titulos, rating y votos
    titles, ratings, votes = data_web(lst_sopa)
    
    
    # Lista auxiliar para identificar de cuál url vienen los datos
    lst_pag = [n_url]*len(titles)
    
    # agregar la información en df_web
    df_web = pd.concat([df_web, pd.DataFrame({'title':titles, 'rating': ratings, 'votes': votes, 'pag': lst_pag})])
    
    # Actualizar el número de registros que hay en el dataframe
    n_data= df_web.shape[0]
    
    # aumentar el contador auxiliar para cambiar de url del url
    n_url+=50

Wall time: 9min 38s


In [70]:
print('Total de registros:', df_web.shape[0])

df_web.head()

Total de registros: 15045


,title,rating,votes,pag
0,The Boys,8.7,205184,1.0
1,Schitt's Creek,8.5,50488,1.0
2,Scam 1992: The Harshad Mehta Story,9.6,37823,1.0
3,Emily en París,7.2,24066,1.0
4,The Mandalorian,8.7,202500,1.0


#### Análisis de duplicados

In [20]:
# Validamos
len(df_web.title.unique())

9299

In [71]:
df_web.title.value_counts().to_frame().head(30)

,title
Utopia,115
Shameless,114
Raised by Wolves,114
Stranger Things,114
The Blacklist,113
Helstrom,113
The Umbrella Academy,113
The Boys,113
The Alienist,113
Grand Army,113


In [72]:
df_web[ df_web.title == 'The Walking Dead']

,title,rating,votes,pag
9,The Walking Dead,8.2,837704,1.0
9,The Walking Dead,8.2,837704,10001.0
9,The Walking Dead,8.2,837704,10051.0
9,The Walking Dead,8.2,837704,10101.0
9,The Walking Dead,8.2,837704,10151.0
...,...,...,...,...
9,The Walking Dead,8.2,837704,15351.0
9,The Walking Dead,8.2,837704,15401.0
9,The Walking Dead,8.2,837704,15451.0
9,The Walking Dead,8.2,837704,15501.0


Observamos que los registros duplicados solamente cambian en la pag que se refiere al url

In [78]:
df_web.drop_duplicates(['title'], inplace=True)

In [79]:
# Eliminamos columna página
df_web.drop(['pag'], axis=1, inplace=True)

### CONSOLIDACIÓN DE LA BDD

Para cruzar las 3 bdd (df_csv, df_api, df_web), se cruzarán por medio de la llave 'title' conservando sólo las coincidencias entre las 3 tablas.

In [145]:
# La llave para realizar los cruces es title
bdd = pd.merge(df_csv, df_api, on='title', how='inner')

In [146]:
# Verificamos duplicados
print('Total de registros en la bdd:', bdd.shape[0])
print('Títulos únicos:', len(bdd.title.unique()))

Total de registros en la bdd: 875
Títulos únicos: 875


In [147]:
bdd = pd.merge(bdd, df_web, on='title', how='inner')

In [148]:
# Verificamos duplicados
print('Total de registros en la bdd:', bdd.shape[0])
print('Títulos únicos:', len(bdd.title.unique()))

Total de registros en la bdd: 531
Títulos únicos: 531


### LIMPIEZA DE LA BDD

In [149]:
print('-------------- Porcentaje de nulos -----------------')
bdd.isna().sum()/bdd.shape[0]*100


-------------- Porcentaje de nulos -----------------


title              0.000000
director          94.161959
cast               6.779661
duration           0.000000
classification     0.188324
listed_in          0.000000
description        0.000000
start_date         0.941620
end_date          96.986817
country            0.000000
network            0.000000
status             0.000000
rating             0.000000
votes              0.000000
dtype: float64

In [150]:
# Análisis de los títulos con status == 'Ended' y 'end_date' vacío
bdd[ (bdd.end_date.isna()) & (bdd.status == 'Ended')].shape[0]/bdd.shape[0]*100

55.55555555555556

Las columnas 'director' y 'end_date' tienen más del 90% de nulos por los que no son columnas que aporten información significativa.

In [151]:
bdd.drop(['director', 'end_date'], axis = 1, inplace=True )

Llenar vacíos de  columnas 'cast' y 'classification' y eliminamos los 4 registros que tienen vacío el campo start_date

In [152]:
bdd['cast'].fillna('Unknown', inplace=True)

bdd['classification'].fillna('Unknown', inplace=True)

In [153]:
# Eliminamos los 4 registros que tienen vacío el campo start_date
bdd = bdd [ bdd.start_date.isna() == False]


Transformamos la columna 'duration' en un campo numérico

In [154]:
bdd['duration'] = bdd['duration'].str.replace('Season', '').str.replace('s', '')

bdd['duration'] = bdd['duration'].str.strip()

bdd['duration'] = bdd['duration'].astype(int)

# Renombramos a seasons
bdd.rename(columns = {'duration':'seasons'}, inplace=True)

In [155]:
# Cambiar tipo de datos
bdd['votes'] = bdd['votes'].astype(int)
bdd['rating'] = bdd['rating'].astype(float)

In [158]:
# Reorganizando las columnas
bdd =  bdd[['title', 'network', 'country', 'classification', 'start_date', 'seasons', 'status', 'rating', 'votes',
            'cast', 'description', 'listed_in']]

Transformar columna 'listed_in' en columnas dummies

In [159]:
type_show = ['TV Comedies', 'Crime TV Shows', 'TV Dramas', 'Docuseries', 'Teen TV Shows', 'TV Horror', "Kids' TV", 
             'Romantic TV Shows', 'Science & Nature TV', 'British TV Shows', 'TV Action & Adventure', 'Anime Series', 
             'Reality TV', 'TV Mysteries', 'TV Sci-Fi & Fantasy']

for t in type_show:
    
    # Generar el nombre de las columnas
    s = t.replace("'", '').replace('&', '').replace('-','').lower()
    lst = s.split()
    s = '_'.join(lst)

    
    # Crear columnas dummies según el tipo de show
    bdd[s] = bdd['listed_in'].apply(lambda x: 1 if t in x else 0)
    
# Eliminamos columna listed_in
bdd.drop(['listed_in'], axis=1, inplace=True)

<ipython-input-159-bb104fffba1a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdd[s] = bdd['listed_in'].apply(lambda x: 1 if t in x else 0)
C:\Users\Itzel\Documents\Software\Anaconda\Anaconda\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [161]:
# Exportar
bdd.to_csv('tv_shows_bdd.csv', index=None)